<a href="https://colab.research.google.com/github/joelma78/GitPY/blob/main/IMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# Notebook completo: Classificação de IMC com Rede Neural (MLP)

# 1. Importação das bibliotecas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from tensorflow.keras.models import Sequential, load_model # Moved load_model here
from tensorflow.keras.layers import Dense
import gradio as gr
import os

# 2. Leitura do dataset
# Substitua o caminho abaixo pelo caminho do seu arquivo dadosimc.csv
dataset_path = '/content/dadosimc.csv'
df = pd.read_csv(dataset_path)
df.head()

# 3. Limpeza e exploração dos dados
# Verificar valores nulos
df.isnull().sum()

# Informações gerais do dataset
df.info()

# Estatísticas descritivas
df.describe()

# 4. Criação da variável alvo (classificação do IMC)
def classificar_imc(imc):
    if imc < 18.5:
        return 'baixo peso'
    elif imc < 25:
        return 'normal'
    elif imc < 30:
        return 'sobrepeso'
    else:
        return 'obesidade'

df['imc_categoria'] = df['imc'].apply(classificar_imc)
df['imc_categoria'].value_counts()

# Filter out rows where peso or altura are 0
df = df[(df['peso'] > 0) & (df['altura'] > 0)].copy()


# 5. Pré-processamento
# Codificar a variável categórica 'sexo'
le_sexo = LabelEncoder()
df['sexo_encoded'] = le_sexo.fit_transform(df['sexo'])

# Encode the target variable
le_imc = LabelEncoder()
le_imc.fit(df['imc_categoria'])


# Selecionar features e alvo
X = df[['idade', 'sexo_encoded', 'peso', 'altura']]
y = df['imc_categoria']


# Divisão em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Normalização das variáveis numéricas
scaler = StandardScaler()
X_train[['idade','peso','altura']] = scaler.fit_transform(X_train[['idade','peso','altura']])
X_test[['idade','peso','altura']] = scaler.transform(X_test[['idade','peso','altura']])

# Transformar y em valores numéricos para a rede neural
y_train_enc = le_imc.transform(y_train)
y_test_enc = le_imc.transform(y_test)

# 6. Construção da rede neural (MLP)
model = Sequential([
    Dense(16, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(len(le_imc.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinamento do modelo
model.fit(X_train, y_train_enc, epochs=30, batch_size=8, verbose=1)

# Salvar o modelo treinado no mesmo diretório do dataset
model_save_path = os.path.join(os.path.dirname(dataset_path), 'modelo_imc_mlp.h5')
model.save(model_save_path)


# 7. Avaliação do modelo
y_pred_enc = np.argmax(model.predict(X_test), axis=1)

print("Acurácia:", accuracy_score(y_test_enc, y_pred_enc))
print("Matriz de Confusão:\n", confusion_matrix(y_test_enc, y_pred_enc))
print("Classification Report:\n", classification_report(y_test_enc, y_pred_enc, target_names=le_imc.classes_))

# 8. App interativo com Gradio

# Salvando o modelo treinado
loaded_model = load_model(model_save_path)

def prever_imc_categoria_nn_loaded(idade, sexo, peso, altura):
    sexo_num = le_sexo.transform([sexo])[0]
    df_temp = pd.DataFrame([[idade, sexo_num, peso, altura]], columns=['idade','sexo_encoded','peso','altura'])
    df_temp[['idade','peso','altura']] = scaler.transform(df_temp[['idade','peso','altura']])
    pred_enc = np.argmax(loaded_model.predict(df_temp), axis=1)[0]
    categoria = le_imc.inverse_transform([pred_enc])[0]
    imc_real = peso / (altura ** 2)
    return f"IMC calculado: {imc_real:.2f}\nCategoria prevista: {categoria}"

gr_interface = gr.Interface(
    fn=prever_imc_categoria_nn_loaded, # Use the new function
    inputs=[
        gr.Number(label="Idade"),
        gr.Dropdown(choices=df['sexo'].unique().tolist(), label="Sexo"),
        gr.Number(label="Peso (kg)"),
        gr.Number(label="Altura (m)")
    ],
    outputs=gr.Textbox(label="Resultado"),
    title="Previsão de Categoria de IMC ",
    description="Entre com seus dados"
)

gr_interface.launch()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   idade   300 non-null    int64  
 1   sexo    300 non-null    object 
 2   peso    300 non-null    float64
 3   altura  300 non-null    float64
 4   imc     300 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 11.8+ KB
Epoch 1/30


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.2655 - loss: 1.3489
Epoch 2/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3891 - loss: 1.2718 
Epoch 3/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4914 - loss: 1.2876 
Epoch 4/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5362 - loss: 1.2317
Epoch 5/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6807 - loss: 1.1134 
Epoch 6/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6738 - loss: 1.0846 
Epoch 7/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6448 - loss: 1.0448 
Epoch 8/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6911 - loss: 1.0171 
Epoch 9/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7210 - loss: 0.9726 
Epoch 10/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6831 - loss: 0.9391 
Epoch 11/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6619 - loss: 0.9599 
Epoch 12/30
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7349 - loss

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Acurácia: 0.8076923076923077
Matriz de Confusão:
 [[15  2  0  0]
 [ 1 11  0  3]
 [ 0  0 10  0]
 [ 0  2  2  6]]
Classification Report:
               precision    recall  f1-score   support

  baixo peso       0.94      0.88      0.91        17
      normal       0.73      0.73      0.73        15
   obesidade       0.83      1.00      0.91        10
   sobrepeso       0.67      0.60      0.63        10

    accuracy                           0.81        52
   macro avg       0.79      0.80      0.80        52
weighted avg       0.81      0.81      0.81        52



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b831a5ab02dc70e0d3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
